In [2]:
# Transkribus login data
username = ""
password = ""

# An outfolder must exist
outfolder = ""

In [45]:
# connect to transkribus
# iterate all pages
# for each page get the pagexml
# get the unicode elements for each line (that we judge relevant?)
# print it in this form to a textfile

# TODO: Fix errors with ü and ä
# TODO: Include pagenumber in filenames
# TODO: Correct misordering of lines if necessary

from transkribusConnect import login, getDocumentR
import xml.etree.ElementTree as et
from pagexml.parser import parse_pagexml_file
import requests
import os
from nltk.tokenize import WordPunctTokenizer

data = login(username, password)
data = et.fromstring(data)
sid = data.find("sessionId").text
# colls = getCollections(sid)

# input collection Id (first arg) and Document Id (second arg)
doc = getDocumentR(169494, 1335480, sid)
for page in doc["pageList"]["pages"]:
    transcript = page["tsList"]["transcripts"][0]
    r = requests.get(transcript["url"])
    if r.status_code == requests.codes.ok:
        text = r.text
    else:
        print(r)
        print("Couldn't read transcript.")
        raise NotImplementedError
    scan = parse_pagexml_file("", pagexml_data=text)
    # open a new textfile named after the pagename
    pagename = page["imgFileName"].replace(".jpg", ".txt")
    outfile = open(os.path.join(outfolder, pagename), mode="w", encoding="utf8")
    textregions = scan.get_text_regions_in_reading_order()
    for textregion in textregions:
        if "type" in textregion.json["metadata"] and textregion.json["metadata"]["type"] == "paragraph":
            lines = textregion.get_lines()
            for line in lines:
                linetext = line.json["text"]
                if linetext:
                    # preprocess linetext by splitting punctuation from words
                    # this also separates numbers, but i think we can live with that
                    # it will be harder to reconnect those though
                    linetext = " ".join(WordPunctTokenizer().tokenize(linetext))

                    outfile.write(linetext + "\n")
    outfile.close()
